Not on all WGS variants, only the union of clumped variants in traits.

See xxxx for all WGS variants QC.

In [ ]:
## import packages
from datetime import datetime
import os
import pandas as pd
import numpy as np
import random
from itertools import chain
import hail as hl
from hail.linalg import BlockMatrix
import statsmodels.api as sm

In [2]:
DATASET = os.getenv('WORKSPACE_CDR')
bucket = os.getenv('WORKSPACE_BUCKET')

In [3]:
# set up hail
hl.init(default_reference = "GRCh38")

/opt/conda/lib/python3.10/site-packages/hail/context.py:352: UserWarning:

Using hl.init with a default_reference argument is deprecated. To set a default reference genome after initializing hail, call `hl.default_reference` with an argument to set the default reference genome.

/opt/conda/lib/python3.10/site-packages/hailtop/aiocloud/aiogoogle/user_config.py:43: UserWarning:

Reading spark-defaults.conf to determine GCS requester pays configuration. This is deprecated. Please use `hailctl config set gcs_requester_pays/project` and `hailctl config set gcs_requester_pays/buckets`.

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.0
SparkUI available at http://all-of-us-11150-m.us-central1-f.c.terra-vpc-sc-fd39b54c.internal:34015
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.130.post1-c69cd67afb8b
LOGGING: writing

In [4]:
# read raw WGS mt
mt_wgs_path = os.getenv("WGS_HAIL_STORAGE_PATH")
mt_wgs = hl.read_matrix_table(mt_wgs_path)
mt_wgs.count()

(702574937, 98590)

In [5]:
# read QC'd variants
var_wgs = hl.read_table(f'{bucket}/WGSData/WGS_Vars_QCed.ht')
var_wgs = var_wgs.key_by("locus")
print(f'{var_wgs.count()} WGS variants passing QC') 

# read QC'd samples
sample_inter = hl.read_table(f'{bucket}/hail_checkpoints/sample_inter.ht')
sample_inter = sample_inter.key_by(sample_inter.person_id)
print(f'{sample_inter.count()} samples passing QC')

8996707 WGS variants passing QC
95562 samples passing QC


In [6]:
# read union clumped variants
union_table = hl.read_table(f'{bucket}/hail_checkpoints/combined_sumstats_union.ht')

union_table_count = union_table.count()
print(f"union_table row count: {union_table_count}")

union_table row count: 2156896


In [ ]:
# subset to QC'd "clumped variants" and samples
mt_wgs = mt_wgs.semi_join_rows(union_table) #this should already QC'd
mt_wgs = mt_wgs.semi_join_cols(sample_inter)

In [ ]:
# retrieve relavent fields
mt_wgs = mt_wgs.select_entries(GT = mt_wgs.GT.n_alt_alleles())
mt_wgs = mt_wgs.select_rows(rsid = mt_wgs.rsid)

In [9]:
###### 20 workers
start = datetime.now() 

In [10]:
# save the WGS MT with QC'ed union clumped variants and samples
mt_wgs.write(f"{bucket}/WGSData/WGS_GT_QCed_clumpedUnion.mt")

2024-10-29 04:33:41.184 Hail: INFO: wrote table with 2156896 rows in 5 partitions to /tmp/__iruid_325-PK6HFfpLz4Yy7PFGeMHOMK
2024-10-29 07:08:20.106 Hail: INFO: wrote matrix table with 2156896 rows and 95562 columns in 140126 partitions to gs://fc-secure-9afe7562-2fad-4781-ab60-03528a626c19/WGSData/WGS_GT_QCed_clumpedUnion.mt


In [11]:
stop = datetime.now()
str(stop-start)

'2:35:04.927608'

In [14]:
wgs_mt.count()

(2156896, 95562)